### Get DC bboxes

In [ ]:
import geopandas as gpd

# Carrega o grid
grid = gpd.read_file('grid/grid_bh.shp')
grid_wgs84 = grid.to_crs(epsg=4326)
bbox_list = []

# Tamanho da bbox ao redor do ponto central (em graus)
buffer_size = 0.001  # ajuste este valor conforme necessário

for _, cell in grid_wgs84.iterrows():
    cell_id = cell['NOARQXWD'] if 'NOARQXWD' in cell else cell.name
    
    # Obtém o ponto central da célula
    centroid = cell.geometry.centroid
    
    # Cria uma bbox ao redor do ponto central
    minx = centroid.x - buffer_size
    maxx = centroid.x + buffer_size
    miny = centroid.y - buffer_size
    maxy = centroid.y + buffer_size
    
    bbox_str = f"[{minx}, {miny}, {maxx}, {maxy}]"
    bbox_list.append((cell_id, bbox_str))
    
with open('centroid_bboxes.txt', 'w') as f:
    for cell_id, bbox_str in bbox_list:
        f.write(f"{cell_id}: {bbox_str}\n")

### Collect Datacubes

In [ ]:
from pystac_client import Client
import odc.stac
import ast
import rioxarray
from requests.exceptions import RequestException
import logging
import os
import json
from pathlib import Path

# Configuração de logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def load_checkpoint(checkpoint_file):
    """Carrega o estado do checkpoint se existir"""
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, 'r') as f:
            return json.load(f)
    return {'completed': [], 'failed': []}

def save_checkpoint(checkpoint_file, state):
    """Salva o estado atual no checkpoint"""
    with open(checkpoint_file, 'w') as f:
        json.dump(state, f)

def get_datacube(catalog_url, bbox, product_name, time_interval, resolution=0.2, bands=["red", "green", "blue"], crs="EPSG:31983"):
    try:
        client = Client.open(catalog_url, timeout=30)
        search = client.search(
            collections=product_name,
            bbox=ast.literal_eval(bbox),
            datetime=time_interval,
        )
        items = search.item_collection()
        
        if not items:
            logger.warning(f"Nenhum item encontrado para {product_name} no bbox {bbox}")
            return None
            
        for item in items:
            for asset_key, asset in item.assets.items():
                if "proj:code" in asset.extra_fields:
                    asset.extra_fields["proj:epsg"] = int(asset.extra_fields["proj:code"].split(":")[1])
        
        return odc.stac.load(
            items,
            bands=bands,
            crs=crs,
            resolution=resolution,
        )
        
    except Exception as e:
        logger.error(f"Erro ao processar {product_name} no bbox {bbox}: {str(e)}")
        return None

In [ ]:
# Configurações
catalog_url = "http://aqui.io/cuborizonte/catalogo/stac"
time_interval = "2015-01-01/2015-12-31"
resolution = 0.2

X_product_name = "bh_ortophoto"
X_bands = ["red", "green", "blue"]

y_product_name = "bh_lidar_rasterized"
y_bands = ["classification"]

X_path = "data/X/"
y_path = "data/y/"
checkpoint_file = "processing_checkpoint.json"


In [ ]:
# Criar diretórios se não existirem
Path(X_path).mkdir(parents=True, exist_ok=True)
Path(y_path).mkdir(parents=True, exist_ok=True)

# Carregar estado anterior
checkpoint = load_checkpoint(checkpoint_file)
completed = set(checkpoint['completed'])
failed_previous_run = set(checkpoint['failed'])

# Processar cada grid
for grid_name, grid_bbox in bbox_list:
    # Pular grids já processados com sucesso
    if grid_name in completed:
        logger.info(f"Pulando {grid_name} - já processado com sucesso")
        continue
        
    logger.info(f"Processando grid: {grid_name}")
    success = True
    
    try:
        # Processar dados X
        x_file = f"{X_path}/{grid_name}.tif"
        if not os.path.exists(x_file):
            xx = get_datacube(catalog_url, grid_bbox, X_product_name, time_interval, resolution, X_bands)
            if xx is not None:
                xx = xx.isel(time=0) 
                xx_array = xx.to_array('band').transpose('band', 'y', 'x')
                xx_array.rio.to_raster(x_file, driver="GTiff")
                logger.info(f"Dados X salvos para {grid_name}")
            else:
                success = False
                logger.warning(f"Falha ao obter dados X para {grid_name}")
        else:
            logger.info(f"Arquivo X já existe para {grid_name}")

        # Processar dados y apenas se X foi bem sucedido
        y_file = f"{y_path}/{grid_name}.tif"
        if success and not os.path.exists(y_file):
            yy = get_datacube(catalog_url, grid_bbox, y_product_name, time_interval, resolution, y_bands)
            if yy is not None:
                yy = yy.isel(time=0)
                yy_array = yy.to_array('band').transpose('band', 'y', 'x')
                yy_array.rio.to_raster(y_file, driver="GTiff")
                logger.info(f"Dados y salvos para {grid_name}")
            else:
                success = False
                logger.warning(f"Falha ao obter dados y para {grid_name}")
        elif os.path.exists(y_file):
            logger.info(f"Arquivo y já existe para {grid_name}")

        # Atualizar checkpoint
        if success:
            completed.add(grid_name)
            if grid_name in failed_previous_run:
                failed_previous_run.remove(grid_name)
            save_checkpoint(checkpoint_file, {
                'completed': list(completed),
                'failed': list(failed_previous_run)
            })
        else:
            failed_previous_run.add(grid_name)
            save_checkpoint(checkpoint_file, {
                'completed': list(completed),
                'failed': list(failed_previous_run)
            })
            
    except Exception as e:
        logger.error(f"Erro crítico ao processar grid {grid_name}: {str(e)}")
        failed_previous_run.add(grid_name)
        save_checkpoint(checkpoint_file, {
            'completed': list(completed),
            'failed': list(failed_previous_run)
        })
        continue

logger.info("Processamento concluído")

### Process Pares

In [ ]:
class_names = {
    0: "Created, never classified",
    1: "Unclassified",
    2: "Ground",
    3: "Low Vegetation",
    4: "Medium Vegetation",
    5: "High Vegetation",
    6: "Building",
    7: "Low Point (Noise)",
    8: "Model Key Point (Mass Point)",
    9: "Water",
    10: "Rail",
    11: "Road Surface",
    12: "Overlap Points",
    13: "Wire - Guard",
    14: "Wire - Conductor",
    15: "Transmission Tower",
    16: "Wire - Connector",
    17: "Bridge Deck",
    18: "High Noise",
    19: "Overhead Structure",
    20: "Ignored Ground"
}



PIPELINE DE PROCESSAMENTO DE MÁSCARAS Y

Esta pipeline realiza a conversão de máscaras de classificação em imagens RGB temáticas,
aplicando filtros para suavização e removendo artefatos indesejados.

Passos principais:
1. Carrega imagens de máscara (.tif) do diretório de entrada
2. Converte as classes em bandas RGB temáticas:
   - Vermelho: Áreas que NÃO são vegetação alta (5) nem construções (6)
   - Verde: Vegetação alta (classe 5)
   - Azul: Construções (classe 6)
3. Aplica filtros de pós-processamento:
   - Filtro de mediana para reduzir ruído (tamanho 10x10)
5. Salva as imagens processadas no diretório de saída em formato RGB

Observações:
- As imagens de saída mantêm a georreferência original
- O filtro de mediana ajuda a suavizar bordas irregulares
- O valor 0 (background) é tratado separadamente na função maskerize_y

In [ ]:
import rasterio
import numpy as np
import os
from scipy.ndimage import gaussian_filter, generic_filter
from scipy.stats import mode
from scipy.ndimage import median_filter
import cv2


def apply_median_filter(img, size=3):
    return median_filter(img, size=size)

def apply_mode_filter(img, size=3):
    return generic_filter(img, lambda x: mode(x).mode, size=size)


def apply_bilateral_filter(img, d=5, sigma_color=50, sigma_space=50):
    return cv2.bilateralFilter(img, d, sigma_color, sigma_space)


def apply_smoothing(img, sigma=1):
    """Aplica um filtro gaussiano para suavizar a imagem."""
    return gaussian_filter(img, sigma=sigma)

def remove_black_pixels(red_band, green_band, blue_band):
    # Encontra pixels onde todas as bandas são 0 (preto)
    black_mask = (red_band == 0) & (green_band == 0) & (blue_band == 0)
    # Seta a banda vermelha para 255 nesses pixels
    red_band[black_mask] = 255
    return red_band, green_band, blue_band

def maskerize_y(img):
    # Banda Azul: Apenas classe 6 (Building)
    blue_band = np.where(img == 6, 255, 0).astype(np.uint8)
    # Banda Verde: Apenas classe 5 (High Vegetation)
    green_band = np.where(img == 5, 255, 0).astype(np.uint8)
    # Banda Vermelha: Tudo que NÃO é verde (5) nem azul (6)
    #red_band = np.where((img != 5) & (img != 6), 255, 0).astype(np.uint8)
    red_band = np.where((img != 5) & (img != 6) & (img != 0), 255, 0).astype(np.uint8)
    
    return red_band, green_band, blue_band

def pipeline_y(input_dir, output_dir, sigma=1):
    os.makedirs(output_dir, exist_ok=True)
    tif_files = [f for f in os.listdir(input_dir) if f.endswith('.tif')]

    for filename in tif_files:
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename)

        with rasterio.open(input_path) as src:
            img = src.read(1)
            profile = src.profile.copy()

        

        # Gera as bandas RGB a partir da imagem suavizada
        red_band, green_band, blue_band = maskerize_y(img)

        profile.update({
            'count': 3,
            'dtype': 'uint8'
        })

        # Aplica o filtro de suavização
        #red_band = apply_smoothing(red_band, sigma=sigma)
        #green_band = apply_smoothing(green_band, sigma=sigma)
        #blue_band = apply_smoothing(blue_band, sigma=sigma)

        #red_band = apply_mode_filter(red_band, size=3)
        #green_band = apply_mode_filter(green_band, size=3)
        #blue_band = apply_mode_filter(blue_band, size=3)

        red_band = apply_median_filter(red_band, size=10)
        green_band = apply_median_filter(green_band, size=10)
        blue_band = apply_median_filter(blue_band, size=10)

        #red_band = apply_bilateral_filter(red_band)
        #green_band = apply_bilateral_filter(green_band)
        #blue_band = apply_bilateral_filter(blue_band)

        # Garante que não haja (0, 0, 0)
        #red_band, green_band, blue_band = remove_black_pixels(red_band, green_band, blue_band)

        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(red_band, 1)
            dst.write(green_band, 2)
            dst.write(blue_band, 3)
            

        print(f"Imagem processada: {filename} (σ={sigma})")

    print("Processamento concluído para todas as imagens!")

# Exemplo de uso:
input_dir = 'data/y/'
output_dir = 'data/processed/y/'
pipeline_y(input_dir, output_dir, sigma=5)

In [ ]:
import os
import rasterio
from rasterio.windows import Window
import numpy as np
from tqdm import tqdm

def calculate_black_pixels(tile):
    """Calcula porcentagem de pixels pretos (0 em todas as bandas)."""
    if tile.ndim == 2:  # Máscaras (1 banda)
        return np.mean(tile == 0)
    else:  # Imagens RGB (3 bandas)
        return np.mean(np.all(tile == 0, axis=0))

def process_image_pair(raster_path, mask_path, output_x_dir, output_y_dir, tile_size=512):
    """Processa um par de imagem/máscara gerando tiles completos."""
    with rasterio.open(raster_path) as raster_src, rasterio.open(mask_path) as mask_src:
        # Determina o número de tiles completos em cada dimensão
        num_rows = min(raster_src.height, mask_src.height) // tile_size
        num_cols = min(raster_src.width, mask_src.width) // tile_size
        
        if num_rows == 0 or num_cols == 0:
            print(f"Imagem {os.path.basename(raster_path)} muito pequena para tiles de {tile_size}x{tile_size}")
            return 0

        base_name = os.path.splitext(os.path.basename(raster_path))[0]
        tile_count = 0

        for row in tqdm(range(num_rows), desc=f"Linhas - {base_name}"):
            for col in range(num_cols):
                # Calcula a posição do tile
                y = row * tile_size
                x = col * tile_size
                window = Window(x, y, tile_size, tile_size)

                try:
                    # Lê os tiles
                    raster_tile = raster_src.read(window=window)
                    mask_tile = mask_src.read(window=window)
                    
                    # Verifica tamanho exato
                    if raster_tile.shape[1:] != (tile_size, tile_size) or \
                       mask_tile.shape[1:] != (tile_size, tile_size):
                        continue
                    
                    # Aplica filtro de pixels pretos
                    if calculate_black_pixels(raster_tile) > 0.05 or \
                       calculate_black_pixels(mask_tile) > 0.05:
                        continue
                    
                    # Incrementa contador e salva
                    tile_count += 1
                    
                    # Salva imagem (X)
                    output_path = os.path.join(output_x_dir, f"{base_name}_tile_{tile_count}.tif")
                    with rasterio.open(output_path, 'w',
                                     driver='GTiff',
                                     height=tile_size,
                                     width=tile_size,
                                     count=raster_src.count,
                                     dtype=raster_tile.dtype,
                                     crs=raster_src.crs,
                                     transform=rasterio.windows.transform(window, raster_src.transform)) as dst:
                        dst.write(raster_tile)
                    
                    # Salva máscara (Y)
                    output_path = os.path.join(output_y_dir, f"{base_name}_tile_{tile_count}.tif")
                    with rasterio.open(output_path, 'w',
                                     driver='GTiff',
                                     height=tile_size,
                                     width=tile_size,
                                     count=mask_src.count,
                                     dtype=mask_tile.dtype,
                                     crs=mask_src.crs,
                                     transform=rasterio.windows.transform(window, mask_src.transform)) as dst:
                        dst.write(mask_tile)
                
                except Exception as e:
                    print(f"Erro no tile {row},{col}: {str(e)}")
                    continue
        
        return tile_count

def process_dataset(raster_dir, mask_dir, output_x_dir, output_y_dir, tile_size=512):
    """Processa todo o conjunto de dados."""
    # Cria diretórios de saída
    os.makedirs(output_x_dir, exist_ok=True)
    os.makedirs(output_y_dir, exist_ok=True)
    
    # Encontra pares correspondentes
    raster_files = {os.path.splitext(f)[0]: f for f in os.listdir(raster_dir) if f.endswith('.tif')}
    mask_files = {os.path.splitext(f)[0]: f for f in os.listdir(mask_dir) if f.endswith('.tif')}
    common_files = sorted(set(raster_files.keys()) & set(mask_files.keys()))
    
    print(f"\n=== Processamento Iniciado ===")
    print(f"Pares encontrados: {len(common_files)}")
    print(f"Tamanho do tile: {tile_size}x{tile_size}")
    print(f"Limite de pixels pretos: 5%\n")
    
    total_tiles = 0
    
    for base_name in common_files:
        try:
            print(f"\nProcessando: {base_name}.tif")
            tiles_gerados = process_image_pair(
                os.path.join(raster_dir, raster_files[base_name]),
                os.path.join(mask_dir, mask_files[base_name]),
                output_x_dir,
                output_y_dir,
                tile_size
            )
            print(f"Tiles gerados: {tiles_gerados}")
            total_tiles += tiles_gerados
        except Exception as e:
            print(e)
    
    print(f"\n=== Processamento Concluído ===")
    print(f"Total de tiles gerados: {total_tiles}")
    print(f"Imagens X salvas em: {output_x_dir}")
    print(f"Máscaras Y salvas em: {output_y_dir}")

# Diretórios de entrada
RASTER_DIR = 'data/X/'          # Imagens RGB
MASK_DIR = 'data/processed/y/'  # Máscaras

# Diretórios de saída
OUTPUT_X_DIR = 'dataset/X/'     # Tiles de imagens
OUTPUT_Y_DIR = 'dataset/y/'     # Tiles de máscaras

# Tamanho do tile (fixo)
TILE_SIZE = 512

# Executa o processamento
process_dataset(RASTER_DIR, MASK_DIR, OUTPUT_X_DIR, OUTPUT_Y_DIR, TILE_SIZE)


=== Processamento Iniciado ===
Pares encontrados: 338
Tamanho do tile: 512x512
Limite de pixels pretos: 5%


Processando: 4243.tif


Linhas - 4243: 100%|██████████| 9/9 [00:04<00:00,  1.82it/s]


Tiles gerados: 1

Processando: 4244.tif


Linhas - 4244: 100%|██████████| 9/9 [00:07<00:00,  1.20it/s]


Tiles gerados: 18

Processando: 4341.tif


...

Linhas - 4735: 100%|██████████| 9/9 [00:10<00:00,  1.18s/it]


Tiles gerados: 17

Processando: 4736.tif


Linhas - 4736: 100%|██████████| 9/9 [00:21<00:00,  2.36s/it]


Tiles gerados: 74

Processando: 4737.tif


Linhas - 4737:  11%|█         | 1/9 [00:01<00:12,  1.61s/it]

### Preenche y com 255 em red

In [ ]:
import rasterio
import numpy as np
import os
from scipy.ndimage import gaussian_filter, generic_filter
from scipy.stats import mode
from scipy.ndimage import median_filter
import cv2


def apply_median_filter(img, size=3):
    return median_filter(img, size=size)

def apply_mode_filter(img, size=3):
    return generic_filter(img, lambda x: mode(x).mode, size=size)


def apply_bilateral_filter(img, d=5, sigma_color=50, sigma_space=50):
    return cv2.bilateralFilter(img, d, sigma_color, sigma_space)


def apply_smoothing(img, sigma=1):
    """Aplica um filtro gaussiano para suavizar a imagem."""
    return gaussian_filter(img, sigma=sigma)

def remove_black_pixels(red_band, green_band, blue_band):
    # Encontra pixels onde todas as bandas são 0 (preto)
    black_mask = (red_band == 0) & (green_band == 0) & (blue_band == 0)
    # Seta a banda vermelha para 255 nesses pixels
    red_band[black_mask] = 255
    return red_band, green_band, blue_band

def maskerize_y(img):
    # Banda Azul: Apenas classe 6 (Building)
    blue_band = np.where(img == 6, 255, 0).astype(np.uint8)
    # Banda Verde: Apenas classe 5 (High Vegetation)
    green_band = np.where(img == 5, 255, 0).astype(np.uint8)
    # Banda Vermelha: Tudo que NÃO é verde (5) nem azul (6)
    #red_band = np.where((img != 5) & (img != 6), 255, 0).astype(np.uint8)
    red_band = np.where((img != 5) & (img != 6) & (img != 0), 255, 0).astype(np.uint8)
    
    return red_band, green_band, blue_band

def pipeline_y(input_dir, output_dir, sigma=1):
    os.makedirs(output_dir, exist_ok=True)
    tif_files = [f for f in os.listdir(input_dir) if f.endswith('.tif')]

    for filename in tif_files:
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename)

        with rasterio.open(input_path) as src:
            img = src.read(1)
            profile = src.profile.copy()

        

        # Gera as bandas RGB a partir da imagem suavizada
        red_band, green_band, blue_band = maskerize_y(img)

        profile.update({
            'count': 3,
            'dtype': 'uint8'
        })

        # Aplica o filtro de suavização
        #red_band = apply_smoothing(red_band, sigma=sigma)
        #green_band = apply_smoothing(green_band, sigma=sigma)
        #blue_band = apply_smoothing(blue_band, sigma=sigma)

        #red_band = apply_mode_filter(red_band, size=3)
        #green_band = apply_mode_filter(green_band, size=3)
        #blue_band = apply_mode_filter(blue_band, size=3)

        red_band = apply_median_filter(red_band, size=10)
        green_band = apply_median_filter(green_band, size=10)
        blue_band = apply_median_filter(blue_band, size=10)

        #red_band = apply_bilateral_filter(red_band)
        #green_band = apply_bilateral_filter(green_band)
        #blue_band = apply_bilateral_filter(blue_band)

        # Garante que não haja (0, 0, 0)
        #red_band, green_band, blue_band = remove_black_pixels(red_band, green_band, blue_band)

        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(red_band, 1)
            dst.write(green_band, 2)
            dst.write(blue_band, 3)
            

        print(f"Imagem processada: {filename} (σ={sigma})")

    print("Processamento concluído para todas as imagens!")

# Exemplo de uso:
input_dir = 'dataset/y/'
output_dir = 'dataset/w/'
pipeline_y(input_dir, output_dir, sigma=5)

In [ ]:
import rasterio
import numpy as np
import os

def remove_black_pixels(red_band, green_band, blue_band):
    # Encontra pixels onde todas as bandas são 0 (preto)
    black_mask = (red_band == 0) & (green_band == 0) & (blue_band == 0)
    # Seta a banda vermelha para 255 nesses pixels
    red_band[black_mask] = 255
    return red_band, green_band, blue_band

def pipeline_y(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    tif_files = [f for f in os.listdir(input_dir) if f.endswith('.tif')]

    for filename in tif_files:
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename)

        with rasterio.open(input_path) as src:
            # Assume the image has 3 bands (RGB)
            red_band = src.read(1)
            green_band = src.read(2)
            blue_band = src.read(3)
            profile = src.profile.copy()

        # Transform black pixels to red
        red_band, green_band, blue_band = remove_black_pixels(red_band, green_band, blue_band)

        profile.update({
            'count': 3,
            'dtype': 'uint8'
        })

        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(red_band, 1)
            dst.write(green_band, 2)
            dst.write(blue_band, 3)

        print(f"Imagem processada: {filename}")

    print("Processamento concluído para todas as imagens!")

input_dir = 'dataset/y/'
output_dir = 'w/'
pipeline_y(input_dir, output_dir)

In [ ]:
import os
import rasterio
from tqdm import tqdm
from rasterio.windows import Window

def process_single_image(raster_path, output_dir, tile_size=512):
    """Processa uma única imagem gerando tiles completos."""
    with rasterio.open(raster_path) as raster_src:
        # Determina o número de tiles completos em cada dimensão
        num_rows = raster_src.height // tile_size
        num_cols = raster_src.width // tile_size
        
        if num_rows == 0 or num_cols == 0:
            print(f"Imagem {os.path.basename(raster_path)} muito pequena para tiles de {tile_size}x{tile_size}")
            return 0

        base_name = os.path.splitext(os.path.basename(raster_path))[0]
        tile_count = 0

        for row in range(num_rows):
            for col in range(num_cols):
                # Calcula a posição do tile
                y = row * tile_size
                x = col * tile_size
                window = Window(x, y, tile_size, tile_size)

                try:
                    # Lê o tile
                    raster_tile = raster_src.read(window=window)
                    
                    # Verifica tamanho exato
                    if raster_tile.shape[1:] != (tile_size, tile_size):
                        continue
                    
                    
                    # Incrementa contador e salva
                    tile_count += 1
                    
                    # Salva imagem
                    output_path = os.path.join(output_dir, f"{base_name}_tile_{tile_count}.tif")
                    with rasterio.open(output_path, 'w',
                                    driver='GTiff',
                                    height=tile_size,
                                    width=tile_size,
                                    count=raster_src.count,
                                    dtype=raster_tile.dtype,
                                    crs=raster_src.crs,
                                    transform=rasterio.windows.transform(window, raster_src.transform)) as dst:
                        dst.write(raster_tile)
                
                except Exception as e:
                    print(f"Erro no tile {row},{col}: {str(e)}")
                    continue
        
        return tile_count

# Configurações
RASTER_DIR = 'X/'
OUTPUT_X_DIR = 'TILES/'
TILE_SIZE = 512

# Certifique-se que o diretório de saída existe
os.makedirs(OUTPUT_X_DIR, exist_ok=True)

# Lista todos os arquivos no diretório RASTER_DIR (ajuste os formatos conforme necessário)
supported_formats = ['.tif', '.tiff', '.geotiff', '.jpg', '.png']
image_files = [f for f in os.listdir(RASTER_DIR) 
              if os.path.splitext(f)[1].lower() in supported_formats]

# Processa cada imagem
total_tiles = 0
for img_file in tqdm(image_files, desc="Processando imagens"):
    img_path = os.path.join(RASTER_DIR, img_file)
    tiles_generated = process_single_image(img_path, OUTPUT_X_DIR, TILE_SIZE)
    total_tiles += tiles_generated
    print(f"{img_file}: {tiles_generated} tiles gerados")

print(f"\nProcessamento concluído! Total de tiles gerados: {total_tiles}")

Processando imagens: 0it [00:00, ?it/s]


Processamento concluído! Total de tiles gerados: 0


In [ ]:
import os
import rasterio
from tqdm import tqdm
import numpy as np
import random

def calculate_black_pixel_percentage(image_path, threshold=0):
    """Calcula a % de pixels pretos na imagem."""
    try:
        with rasterio.open(image_path) as src:
            data = src.read()
            
            # Conta pixels pretos (considerando todas as bandas)
            if len(data.shape) == 3:  # Imagem multibanda
                black_pixels = np.sum(np.all(data <= threshold, axis=0))
            else:  # Banda única
                black_pixels = np.sum(data <= threshold)
            
            total_pixels = data[0].size if len(data.shape) == 3 else data.size
            return (black_pixels / total_pixels) * 100 


# Função probabilíostica para descartar 98% das imagens com mais de 95% de pixels pretos
def should_discard_image(black_percentage):
    """Retorna True com 98% de chance se >5% dos pixels forem pretos."""
    if black_percentage > 5:
        return random.random() < 0.98
    return False


def delete_black_images(tiles_dir, threshold=0):
    """Deleta imagens completamente pretas no diretório especificado."""
    # Lista todos os arquivos no diretório (ajuste os formatos conforme necessário)
    supported_formats = ['.tif', '.tiff', '.geotiff', '.jpg', '.png']
    image_files = [f for f in os.listdir(tiles_dir) 
                 if os.path.splitext(f)[1].lower() in supported_formats]
    
    deleted_count = 0
    for img_file in tqdm(image_files, desc="Verificando imagens pretas"):
        img_path = os.path.join(tiles_dir, img_file)
        black_percentage = calculate_black_pixel_percentage(img_path, threshold)
        if should_discard_image(black_percentage):
            try:
                os.remove(img_path)
                deleted_count += 1
            except Exception as e:
                print(f"Erro ao deletar {img_path}: {str(e)}")
    
    print(f"\nProcessamento concluído! Total de imagens pretas deletadas: {deleted_count}")

# Configurações
TILES_DIR = 'TILES/'
THRESHOLD = 0  # Ajuste este valor conforme necessário

# Executa a limpeza
delete_black_images(TILES_DIR, THRESHOLD)